In [1]:
#Bibliotecas
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

#Estilizar conteúdo
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def estilo_tabelas(df, max_altura='300px', casas_decimais=3):
    def cor_linhas(row):
        if row.name % 2 == 0:
            return ['background-color: #ffffff'] * len(row)
        else:
            return ['background-color: #f9f9f9'] * len(row)
    
    return (
        df.style.apply(cor_linhas, axis=1) 
        .set_table_styles(
            [
                {'selector': 'thead th', 'props': [('font-size', '12px'), ('text-align', 'center'), ('border-bottom', '2px solid #007BFF')]},  # Azul abaixo do nome das colunas
                {'selector': 'td', 'props': [('font-size', '10px'), ('text-align', 'center'), ('max-height', '40px'), ('white-space', 'nowrap'), ('text-overflow', 'ellipsis'), ('overflow', 'hidden'), ('max-width', '300px')]},
                {'selector': 'table', 'props': [('width', '90%'), ('margin-left', 'auto'), ('margin-right', 'auto'), ('border-collapse', 'collapse')]},
                {'selector': 'td, th', 'props': [('border', '1px solid #666')]},
            ]
        )
        .set_properties(
            **{'border-color': 'darkgray', 'border-style': 'solid', 'border-width': '1px'}
        )
        .set_table_attributes(
            f'style="height:auto; overflow:auto; max-height:{max_altura}; display:block;"'  
        )
        .format(
            precision=casas_decimais  
        )
    )

In [2]:
object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

df_eventos = pd.read_csv(
    "https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_desastres_rec_nrec.csv",
    sep=';',
    dtype = dtype,
    decimal=',',
)

estilo_tabelas(df_eventos.head(5))

,Protocolo_S2iD,Nome_Municipio,Sigla_UF,regiao,Data_Registro,Data_Evento,codigo_ibge,Setores Censitários,Status,DH_Descricao,DH_MORTOS,DH_FERIDOS,DH_ENFERMOS,DH_DESABRIGADOS,DH_DESALOJADOS,DH_DESAPARECIDOS,DH_OUTROS AFETADOS,DH_total_danos_humanos,DM_Descricao,DM_Uni Habita Danificadas,DM_Uni Habita Destruidas,DM_Uni Habita Valor,DM_Inst Saúde Danificadas,DM_Inst Saúde Destruidas,DM_Inst Saúde Valor,DM_Inst Ensino Danificadas,DM_Inst Ensino Destruidas,DM_Inst Ensino Valor,DM_Inst Serviços Danificadas,DM_Inst Serviços Destruidas,DM_Inst Serviços Valor,DM_Inst Comuni Danificadas,DM_Inst Comuni Destruidas,DM_Inst Comuni Valor,DM_Obras de Infra Danificadas,DM_Obras de Infra Destruidas,DM_Obras de Infra Valor,DM_total_danos_materiais,DA_Descricao,DA_Polui/cont da água,DA_Polui/cont do ar,DA_Polui/cont do solo,DA_Dimi/exauri hídrico,DA_Incêndi parques/APA's/APP's,PEPL_Descricao,PEPL_Assis_méd e emergên(R$),PEPL_Abast de água pot(R$),PEPL_sist de esgotos sanit(R$),PEPL_Sis limp e rec lixo (R$),PEPL_Sis cont pragas (R$),PEPL_distrib energia (R$),PEPL_Telecomunicações (R$),PEPL_Tran loc/reg/l_curso (R$),PEPL_Distrib combustíveis(R$),PEPL_Segurança pública (R$),PEPL_Ensino (R$),PEPL_total_publico,PEPR_Descricao,PEPR_Agricultura (R$),PEPR_Pecuária (R$),PEPR_Indústria (R$),PEPR_Comércio (R$),PEPR_Serviços (R$),PEPR_total_privado,PE_PLePR,Ano_Evento,Empenhado,DensidadePop,Area,Município - UF,PIB,DOMICILIO_AREARURAL,PDEFAGUA,PDEFESGOTO,PDEFLIXO,PDEFSAN,QTDE_FAMILIAS_ATUALIZADAS,Categoria,Grupo,Subgrupo,Tipo,Subtipo,COBRADE,Pop
0,SP-A-3550001-12200-20100101,São Luiz do Paraitinga,SP,Sudeste,2010-01-01,2010-01-01,3550001,nan,Reconhecido,nan,0,0,0,93,4030,0,16,4139,nan,49,20,20643486.932,3.000,0,1121928.638,9.000,1,5235666.976,0,0,0.000,3,3,32411.272,25.000,0.000,65321178.458,92354672.275,nan,nan,nan,nan,nan,nan,nan,7978.159,673157.183,623293.688,2493.175,0.000,330993.880,3739.762,58609.552,0.000,0.000,1087024.191,2787289.589,nan,1642902.433,1149353.560,379785.310,0.000,24433.113,3196474.416,5983764.005,2010,nan,nan,nan,São Luiz do Paraitinga - SP,86448.000,nan,nan,nan,nan,nan,nan,Natural,Hidrológico,Enxurradas,nan,nan,12200,10397.000
1,SP-A-3518305-11321-20100101,Guararema,SP,Sudeste,2010-01-01,2010-01-01,3518305,nan,Reconhecido,nan,4,2,0,66,417,0,14430,14919,nan,161,91,18604069.987,0.000,0,0.000,2.000,0,24931.748,0,0,0.000,0,0,37397.621,9000.000,0.000,57771845.314,76438244.669,nan,nan,nan,nan,nan,nan,nan,0.000,179508.582,264276.524,12465.874,0.000,349044.465,74795.243,134631.437,0.000,0.000,77288.417,1092010.541,nan,441291.931,2049389.645,0.000,0.000,112192.889,2602874.464,3694885.005,2010,nan,nan,nan,Guararema - SP,1111405.000,nan,nan,nan,nan,nan,nan,Natural,Geológico,Movimento de massa,Deslizamentos,Deslizamentos de solo e ou rocha,11321,25844.000
2,BA-A-2905909-14110-20100101,Campo Alegre de Lourdes,BA,Nordeste,2010-01-01,2010-01-01,2905909,nan,Reconhecido,nan,0,0,0,0,0,0,0,0,nan,0,0,0.000,0.000,0,0.000,0.000,0,0.000,0,0,0.000,0,0,0.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,747.952,448771.455,0.000,0.000,0.000,0.000,0.000,249317.475,0.000,0.000,177015.407,875852.290,nan,254303.825,147097.310,0.000,0.000,0.000,401401.135,1277253.425,2010,nan,9.638,2914.587,Campo Alegre de Lourdes - BA,107459.000,nan,nan,nan,nan,nan,nan,Natural,Climatológico,Seca,Estiagem,nan,14110,28090.000
3,SP-A-3513603-12100-20100101,Cunha,SP,Sudeste,2010-01-01,2010-01-01,3513603,nan,Reconhecido,nan,6,0,0,38,492,0,12000,12536,nan,89,9,3340854.165,0.000,0,0.000,0.000,0,0.000,0,0,0.000,0,0,0.000,3.000,400.000,8052097.614,11392951.779,nan,nan,nan,nan,nan,nan,nan,0.000,1246.587,0.000,0.000,0.000,1994539.800,0.000,125656.007,0.000,0.000,0.000,2121442.395,nan,49863.495,1383711.986,0.000,0.000,925466.467,2359041.949,4480484.344,2010,nan,15.538,1407.250,Cunha - SP,112225.000,nan,nan,nan,nan,nan,nan,Natural,Hidrológico,Inundações,nan,nan,12100,21866.000
4,BA-P-2917334-12200-20100101,Iuiu,BA,Nordeste,2010-01-01,2010-01-01,2917334,nan,Reconhecido,nan,0,0,0,0,0,0,0,0,

In [3]:
#Criar a variável Rendapercapita como proxy para a renda do município

df_eventos['Rendapercapita'] = df_eventos['PIB'] / df_eventos['Pop']

#Excluir variáveis que não serão utilizadas nos modelos

df_eventos = df_eventos.drop(
    ['Protocolo_S2iD', 'Nome_Municipio', 'Data_Registro', 'Data_Evento', 'Município - UF', 'codigo_ibge', 'DH_Descricao', 'DM_Descricao', 'DA_Descricao', 'PEPL_Descricao', 'PEPR_Descricao',
     'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo', 'Setores Censitários', 'Area', 'PIB', 'Pop', 'PE_PLePR'], axis=1)

In [4]:
df_na_counts = df_eventos.isna().sum().reset_index().rename(columns={0: 'Qtde. dados ausentes', 'index': 'Variável'})
df_na_counts['% de dados ausentes'] = (df_na_counts['Qtde. dados ausentes'] / len(df_eventos)) * 100
colunas_mantidas = df_na_counts[df_na_counts['% de dados ausentes'] < 50]['Variável']
df_eventos = df_eventos[colunas_mantidas]
print("Colunas mantidas: ", df_eventos.columns)

Colunas mantidas:  Index(['Sigla_UF', 'regiao', 'Status', 'DH_MORTOS', 'DH_FERIDOS',
       'DH_ENFERMOS', 'DH_DESABRIGADOS', 'DH_DESALOJADOS', 'DH_DESAPARECIDOS',
       'DH_OUTROS AFETADOS', 'DH_total_danos_humanos',
       'DM_Uni Habita Danificadas', 'DM_Uni Habita Destruidas',
       'DM_Uni Habita Valor', 'DM_Inst Saúde Danificadas',
       'DM_Inst Saúde Destruidas', 'DM_Inst Saúde Valor',
       'DM_Inst Ensino Danificadas', 'DM_Inst Ensino Destruidas',
       'DM_Inst Ensino Valor', 'DM_Inst Serviços Danificadas',
       'DM_Inst Serviços Destruidas', 'DM_Inst Serviços Valor',
       'DM_Inst Comuni Danificadas', 'DM_Inst Comuni Destruidas',
       'DM_Inst Comuni Valor', 'DM_Obras de Infra Danificadas',
       'DM_Obras de Infra Destruidas', 'DM_Obras de Infra Valor',
       'DM_total_danos_materiais', 'PEPL_Assis_méd e emergên(R$)',
       'PEPL_Abast de água pot(R$)', 'PEPL_sist de esgotos sanit(R$)',
       'PEPL_Sis limp e rec lixo (R$)', 'PEPL_Sis cont pragas (R$)',
    

In [5]:
encoder = OneHotEncoder(sparse_output=False, drop=None)  
categorias = ['Sigla_UF', 'regiao', 'COBRADE']
one_hot_encoded = encoder.fit_transform(df_eventos[categorias])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorias))

df_eventos = pd.concat([df_eventos.drop(columns=categorias), one_hot_df], axis=1)

In [6]:
df_eventos['Status'] = df_eventos['Status'].map({'Reconhecido': 0, 'Não reconhecido': 1})

In [7]:
df_eventos.head(5)

,Status,DH_MORTOS,DH_FERIDOS,DH_ENFERMOS,DH_DESABRIGADOS,DH_DESALOJADOS,DH_DESAPARECIDOS,DH_OUTROS AFETADOS,DH_total_danos_humanos,DM_Uni Habita Danificadas,DM_Uni Habita Destruidas,DM_Uni Habita Valor,DM_Inst Saúde Danificadas,DM_Inst Saúde Destruidas,DM_Inst Saúde Valor,DM_Inst Ensino Danificadas,DM_Inst Ensino Destruidas,DM_Inst Ensino Valor,DM_Inst Serviços Danificadas,DM_Inst Serviços Destruidas,DM_Inst Serviços Valor,DM_Inst Comuni Danificadas,DM_Inst Comuni Destruidas,DM_Inst Comuni Valor,DM_Obras de Infra Danificadas,DM_Obras de Infra Destruidas,DM_Obras de Infra Valor,DM_total_danos_materiais,PEPL_Assis_méd e emergên(R$),PEPL_Abast de água pot(R$),PEPL_sist de esgotos sanit(R$),PEPL_Sis limp e rec lixo (R$),PEPL_Sis cont pragas (R$),PEPL_distrib energia (R$),PEPL_Telecomunicações (R$),PEPL_Tran loc/reg/l_curso (R$),PEPL_Distrib combustíveis(R$),PEPL_Segurança pública (R$),PEPL_Ensino (R$),PEPL_total_publico,PEPR_Agricultura (R$),PEPR_Pecuária (R$),PEPR_Indústria (R$),PEPR_Comércio (R$),PEPR_Serviços (R$),PEPR_total_privado,Ano_Evento,DensidadePop,DOMICILIO_AREARURAL,PDEFAGUA,PDEFESGOTO,PDEFLIXO,PDEFSAN,QTDE_FAMILIAS_ATUALIZADAS,Rendapercapita,Sigla_UF_AC,Sigla_UF_AL,Sigla_UF_AM,Sigla_UF_AP,Sigla_UF_BA,Sigla_UF_CE,Sigla_UF_DF,Sigla_UF_ES,Sigla_UF_GO,Sigla_UF_MA,Sigla_UF_MG,Sigla_UF_MS,Sigla_UF_MT,Sigla_UF_PA,Sigla_UF_PB,Sigla_UF_PE,Sigla_UF_PI,Sigla_UF_PR,Sigla_UF_RJ,Sigla_UF_RN,Sigla_UF_RO,Sigla_UF_RR,Sigla_UF_RS,Sigla_UF_SC,Sigla_UF_SE,Sigla_UF_SP,Sigla_UF_TO,Sigla_UF_pa,regiao_Centro-oeste,regiao_Nordeste,regiao_Norte,regiao_Sudeste,regiao_Sul,COBRADE_11110,COBRADE_11120,COBRADE_11311,COBRADE_11312,COBRADE_11313,COBRADE_11321,COBRADE_11331,COBRADE_11332,COBRADE_11340,COBRADE_11410,COBRADE_11420,COBRADE_11431,COBRADE_11432,COBRADE_11433,COBRADE_12100,COBRADE_12200,COBRADE_12300,COBRADE_13111,COBRADE_13112,COBRADE_13120,COBRADE_13211,COBRADE_13212,COBRADE_13213,COBRADE_13214,COBRADE_13215,COBRADE_13310,COBRADE_13321,COBRADE_13322,COBRADE_14110,COBRADE_14120,COBRADE_14131,COBRADE_14132,COBRADE_14140,COBRADE_15110,COBRADE_15120,COBRADE_15130,COBRADE_15210,COBRADE_15230,COBRADE_22210,COBRADE_22220,COBRADE_23120,COBRADE_24100,COBRADE_24200,COBRADE_25100,COBRADE_25500
0,0,0,0,0,93,4030,0,16,4139,49,20,2.064349e+07,3.0,0,1.121929e+06,9.0,1,5.235667e+06,0,0,0.0,3,3,32411.271754,25.0,0.0,6.532118e+07,9.235467e+07,7978.159201,673157.182580,623293.687574,2493.174750,0.0,3.309939e+05,3739.762125,58609.552030,0.0,0.0,1.087024e+06,2.787290e+06,1.642902e+06,1.149354e+06,379785.309713,0.0,24433.112553,3.196474e+06,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.314706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,4,2,0,66,417,0,14430,14919,161,91,1.860407e+07,0.0,0,0.000000e+00,2.0,0,2.493175e+04,0,0,0.0,0,0,37397.621254,9000.0,0.0,5.777185e+07,7.643824e+07,0.000000,179508.582021,264276.523531,12465.873751,0.0,3.490445e+05,74795.242509,134631.436516,0.0,0.0,7.728842e+04,1.092011e+06,4.412919e+05,2.049390e+06,0.000000,0.0,112192.888695,2.602874e+06,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.004372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0.000000e+00,0.0,0,0.000000e+00,0.0,0,0.000000e+00,0,0,0.0,0,0,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,747.952425,448771.455053,0.000000,0.000000,0.0,0.000000e+00,0.000000,249317.475030,0.0,0.0,1.770154e+05,8.758523e+05,2.543038e+05,1.470973e+05,0.000000,0.0,0.000000,4.014011e+05,2010,9.637729,NaN,NaN,NaN,NaN,NaN,NaN,3.825525,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0

In [8]:
df_eventos = df_eventos.sample(frac=1, random_state=1).reset_index(drop=True)

In [9]:
df_eventos_2010_2017 = df_eventos[df_eventos['Ano_Evento'] <= 2017]
cadunico = ['DOMICILIO_AREARURAL', 'PDEFAGUA', 'PDEFESGOTO', 'PDEFLIXO', 'PDEFSAN', 'QTDE_FAMILIAS_ATUALIZADAS']
df_eventos_2010_2017 = df_eventos_2010_2017.drop(columns=cadunico)

print("Tamanho da base (2010 a 2017): ", df_eventos_2010_2017.shape)
print("Dados ausentes na base (2010 a 2017) \n", df_eventos_2010_2017.isna().sum())

Tamanho da base (2010 a 2017):  (13047, 127)
Dados ausentes na base (2010 a 2017) 
 Status                               0
DH_MORTOS                            0
DH_FERIDOS                           0
DH_ENFERMOS                          0
DH_DESABRIGADOS                      0
DH_DESALOJADOS                       0
DH_DESAPARECIDOS                     0
DH_OUTROS AFETADOS                   0
DH_total_danos_humanos               0
DM_Uni Habita Danificadas            0
DM_Uni Habita Destruidas             0
DM_Uni Habita Valor                  0
DM_Inst Saúde Danificadas            0
DM_Inst Saúde Destruidas             0
DM_Inst Saúde Valor                  0
DM_Inst Ensino Danificadas           0
DM_Inst Ensino Destruidas            0
DM_Inst Ensino Valor                 0
DM_Inst Serviços Danificadas         0
DM_Inst Serviços Destruidas          0
DM_Inst Serviços Valor               0
DM_Inst Comuni Danificadas           0
DM_Inst Comuni Destruidas            0
DM_Inst Comuni Valo

In [10]:
df_eventos_2018_2024 = df_eventos[(df_eventos['Ano_Evento'] >= 2018)]
df_eventos_2018_2024 = df_eventos_2018_2024.drop(columns="Rendapercapita")

print("Tamanho da base (2018 a 2024): ", df_eventos_2018_2024.shape)
print("Dados ausentes na base (2018 a 2024) \n", df_eventos_2018_2024.isna().sum())

Tamanho da base (2018 a 2024):  (13211, 132)
Dados ausentes na base (2018 a 2024) 
 Status                               0
DH_MORTOS                            0
DH_FERIDOS                           0
DH_ENFERMOS                          0
DH_DESABRIGADOS                      0
DH_DESALOJADOS                       0
DH_DESAPARECIDOS                     0
DH_OUTROS AFETADOS                   0
DH_total_danos_humanos               0
DM_Uni Habita Danificadas            0
DM_Uni Habita Destruidas             0
DM_Uni Habita Valor                  0
DM_Inst Saúde Danificadas            0
DM_Inst Saúde Destruidas             0
DM_Inst Saúde Valor                  0
DM_Inst Ensino Danificadas           0
DM_Inst Ensino Destruidas            0
DM_Inst Ensino Valor                 0
DM_Inst Serviços Danificadas         0
DM_Inst Serviços Destruidas          0
DM_Inst Serviços Valor               0
DM_Inst Comuni Danificadas           0
DM_Inst Comuni Destruidas            0
DM_Inst Comuni Valo

In [11]:
colunas_modelo1 = [
    'DH_total_danos_humanos',
    'DM_total_danos_materiais',
    'PEPL_total_publico',
    'PEPR_total_privado',
    'regiao_Centro-oeste',
    'regiao_Nordeste',
    'regiao_Norte',
    'regiao_Sudeste',
    'regiao_Sul',
    'COBRADE_11110',
    'COBRADE_11120',
    'COBRADE_11311',
    'COBRADE_11312',
    'COBRADE_11313',
    'COBRADE_11321',
    'COBRADE_11331',
    'COBRADE_11332',
    'COBRADE_11340',
    'COBRADE_11410',
    'COBRADE_11420',
    'COBRADE_11431',
    'COBRADE_11432',
    'COBRADE_11433',
    'COBRADE_12100',
    'COBRADE_12200',
    'COBRADE_12300',
    'COBRADE_13111',
    'COBRADE_13112',
    'COBRADE_13120',
    'COBRADE_13211',
    'COBRADE_13212',
    'COBRADE_13213',
    'COBRADE_13214',
    'COBRADE_13215',
    'COBRADE_13310',
    'COBRADE_13321',
    'COBRADE_13322',
    'COBRADE_14110',
    'COBRADE_14120',
    'COBRADE_14131',
    'COBRADE_14132',
    'COBRADE_14140',
    'COBRADE_15110',
    'COBRADE_15120',
    'COBRADE_15130',
    'COBRADE_15210',
    'COBRADE_15230',
    'COBRADE_22210',
    'COBRADE_22220',
    'COBRADE_23120',
    'COBRADE_24100',
    'COBRADE_24200',
    'COBRADE_25100',
    'COBRADE_25500'
]

colunas_CadUnico = [
    'DOMICILIO_AREARURAL',
    'PDEFAGUA',
    'PDEFESGOTO',
    'PDEFLIXO',
    'PDEFSAN',
    'QTDE_FAMILIAS_ATUALIZADAS'
]

colunas_Renda = [
    'Rendapercapita'
]

df_eventos_2010_2017_modelo1 = df_eventos_2010_2017[colunas_modelo1 + colunas_Renda]

df_eventos_2018_2024_modelo1 = df_eventos_2018_2024[colunas_modelo1 + colunas_CadUnico]



In [12]:
print("Dimensões da base de dados 2010-2017, modelo 1: ", df_eventos_2010_2017_modelo1.shape)
estilo_tabelas(df_eventos_2010_2017_modelo1.head(5))

Dimensões da base de dados 2010-2017, modelo 1:  (13047, 55)


,DH_total_danos_humanos,DM_total_danos_materiais,PEPL_total_publico,PEPR_total_privado,regiao_Centro-oeste,regiao_Nordeste,regiao_Norte,regiao_Sudeste,regiao_Sul,COBRADE_11110,COBRADE_11120,COBRADE_11311,COBRADE_11312,COBRADE_11313,COBRADE_11321,COBRADE_11331,COBRADE_11332,COBRADE_11340,COBRADE_11410,COBRADE_11420,COBRADE_11431,COBRADE_11432,COBRADE_11433,COBRADE_12100,COBRADE_12200,COBRADE_12300,COBRADE_13111,COBRADE_13112,COBRADE_13120,COBRADE_13211,COBRADE_13212,COBRADE_13213,COBRADE_13214,COBRADE_13215,COBRADE_13310,COBRADE_13321,COBRADE_13322,COBRADE_14110,COBRADE_14120,COBRADE_14131,COBRADE_14132,COBRADE_14140,COBRADE_15110,COBRADE_15120,COBRADE_15130,COBRADE_15210,COBRADE_15230,COBRADE_22210,COBRADE_22220,COBRADE_23120,COBRADE_24100,COBRADE_24200,COBRADE_25100,COBRADE_25500,Rendapercapita
5,4000,166542.286,6120428.994,7286224.972,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,17.934
8,8034,0.000,0.000,7639477.192,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.605
11,2732,0.000,0.000,26406.490,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,13.655
13,0,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.997
17,0,2048392.375,2617.833,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,10.594


In [13]:
print("Dimensões da base de dados 2018-2024, modelo 1: ", df_eventos_2018_2024_modelo1.shape)
estilo_tabelas(df_eventos_2018_2024_modelo1.head(5))

Dimensões da base de dados 2018-2024, modelo 1:  (13211, 60)


,DH_total_danos_humanos,DM_total_danos_materiais,PEPL_total_publico,PEPR_total_privado,regiao_Centro-oeste,regiao_Nordeste,regiao_Norte,regiao_Sudeste,regiao_Sul,COBRADE_11110,COBRADE_11120,COBRADE_11311,COBRADE_11312,COBRADE_11313,COBRADE_11321,COBRADE_11331,COBRADE_11332,COBRADE_11340,COBRADE_11410,COBRADE_11420,COBRADE_11431,COBRADE_11432,COBRADE_11433,COBRADE_12100,COBRADE_12200,COBRADE_12300,COBRADE_13111,COBRADE_13112,COBRADE_13120,COBRADE_13211,COBRADE_13212,COBRADE_13213,COBRADE_13214,COBRADE_13215,COBRADE_13310,COBRADE_13321,COBRADE_13322,COBRADE_14110,COBRADE_14120,COBRADE_14131,COBRADE_14132,COBRADE_14140,COBRADE_15110,COBRADE_15120,COBRADE_15130,COBRADE_15210,COBRADE_15230,COBRADE_22210,COBRADE_22220,COBRADE_23120,COBRADE_24100,COBRADE_24200,COBRADE_25100,COBRADE_25500,DOMICILIO_AREARURAL,PDEFAGUA,PDEFESGOTO,PDEFLIXO,PDEFSAN,QTDE_FAMILIAS_ATUALIZADAS
0,11650,0.000,132466.926,861035.020,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.305,0.191,0.559,0.167,0.091,383.000
1,2548,0.000,103709.315,6509159.048,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.632,0.246,0.456,0.175,0.070,57.000
2,336,2296093.386,1928129.702,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.708,0.691,0.851,0.691,0.576,356.000
3,420,9811880.000,8974880.000,29950000.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.187,0.098,0.992,0.139,0.077,632.000
4,13245,0.000,286229.997,1353851.954,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.242,0.157,0.187,0.063,0.028,2231.000


In [14]:
colunas_remover_modelo2 = [
    'DH_total_danos_humanos',
    'DM_total_danos_materiais',
    'PEPL_total_publico',
    'PEPR_total_privado',
    'regiao_Centro-oeste',
    'regiao_Nordeste',
    'regiao_Norte',
    'regiao_Sudeste',
    'regiao_Sul'
]

df_eventos_2010_2017_modelo2 = df_eventos_2010_2017.drop(columns=colunas_remover_modelo2)
df_eventos_2010_2017_modelo2 = df_eventos_2010_2017_modelo2[colunas_Renda + df_eventos_2010_2017_modelo2.columns.tolist()]

df_eventos_2018_2024_modelo2 = df_eventos_2018_2024.drop(columns=colunas_remover_modelo2)
df_eventos_2018_2024_modelo2 = df_eventos_2018_2024_modelo2[colunas_CadUnico + df_eventos_2018_2024_modelo2.columns.tolist()]



In [15]:
print("Dimensões da base de dados 2010-2017, modelo 2: ", df_eventos_2010_2017_modelo2.shape)
estilo_tabelas(df_eventos_2010_2017_modelo2.head(5))

Dimensões da base de dados 2010-2017, modelo 2:  (13047, 119)


KeyError: '`Styler.apply` and `.map` are not compatible with non-unique index or columns.'

In [16]:
print("Dimensões da base de dados 2018-2024, modelo 2: ", df_eventos_2018_2024_modelo2.shape)
estilo_tabelas(df_eventos_2018_2024_modelo2.head(5))

Dimensões da base de dados 2018-2024, modelo 2:  (13211, 129)


KeyError: '`Styler.apply` and `.map` are not compatible with non-unique index or columns.'

In [17]:
df_eventos_2010_2017_modelo1.to_csv('df_eventos_2010_2017_modelo1.csv',
                        index=False,
                        sep=';',
                        decimal=',')

df_eventos_2018_2024_modelo1.to_csv('df_eventos_2018_2024_modelo1.csv',
                        index=False,
                        sep=';',
                        decimal=',')

df_eventos_2018_2024_modelo2.to_csv('df_eventos_2018_2024_modelo2.csv',
                        index=False,
                        sep=';',
                        decimal=',')

df_eventos_2010_2017_modelo2.to_csv('df_eventos_2010_2017_modelo2.csv',
                        index=False,
                        sep=';',
                        decimal=',')